In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame

In [2]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [3]:
# https://dspace.library.uu.nl/bitstream/handle/1874/32937/NWS-E-2008-112.pdf?sequence=1

In [4]:
# EUR inflation adjustment
# https://www.ecb.europa.eu/stats/ecb_statistics/escb/html/table.en.html?id=JDF_ICP_COICOP_INX
hicp = pd.read_csv('inflation/eurostat.csv', usecols=['TIME_PERIOD','OBS_VALUE','TITLE_COMPL'])

drop_idx = []
for idx in range(len(hicp)):
    date = hicp['TIME_PERIOD'].iloc[idx]
    if '-01' not in date:
        drop_idx.append(idx)
hicp.drop(drop_idx, inplace=True)

hicp = hicp[hicp['TITLE_COMPL']=='Euro area (changing composition) - HICP - Overall index, Monthly Index, Eurostat, Neither seasonally nor working day adjusted']

years = []
for year in hicp['TIME_PERIOD']:
    years.append(year[:4])
hicp['Year'] = years
hicp.drop(columns=['TIME_PERIOD','TITLE_COMPL'],inplace=True)
hicp.set_index('Year', drop=True, inplace=True)
hicp

,OBS_VALUE
Year,
1996,70.97
1997,72.42
1998,73.20
1999,73.76
2000,75.13
2001,76.63
2002,78.63
2003,80.30
2004,81.75


In [5]:
eur_2006_2022 = float(hicp.loc['2022'] / hicp.loc['2006'])
eur_2006_2022

1.2970123022847102

In [6]:
# convert 2022 EUR to 2022 USD
# https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html

usd_per_1_eur = 1.0530

In [7]:
cfl = pd.read_excel('raw data/cost data/weiss et al.xlsx', usecols=[0,1,4,6], index_col=0)
cfl['Price in EUR2006/kWh'] = cfl['CFL - Price in EUR2006'] / ((cfl['CFL - Average power in W'] * cfl['CFL - Average life time in h']) / 1000)
cfl.drop(columns=['CFL - Price in EUR2006','CFL - Average life time in h','CFL - Average power in W'], inplace=True)
cfl = cfl.transpose()
# inflation adjsutment from 2006 EUR to 2022 EUR and conversion factor from 2022 EUR to 2022 USD
cfl.iloc[0] = cfl.iloc[0] * eur_2006_2022 * usd_per_1_eur
cfl['Unit'] = '2022 USD/kWh'
cfl['Data Source'] = 'Weiss et al.'
cfl['Spatial Scale'] = 'Global'
cfl['Country Code'] = 'World'
cfl['Country Name'] = 'World'
cfl['Metric'] = 'Price'
cfl['Technology Name'] = 'CFLs'
cfl['ID'] = cfl['Technology Name'] + '_' + cfl['Metric'] + '_' + cfl['Country Code']
cfl.set_index('ID', drop=True, inplace=True)
cfl


,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2004,2005,2006,Unit,Data Source,Spatial Scale,Country Code,Country Name,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
CFLs_Price_World,0.204321,0.18481,0.212615,0.242036,0.272447,0.297074,0.282699,0.255404,0.265368,0.243885,...,0.071036,0.07337,0.075723,2022 USD/kWh,Weiss et al.,Global,World,World,Price,CFLs


In [8]:
cfl.to_csv('cleaned data/cfl_prices.csv')